How to create a Deployment
==========================

In this notebook, we show you how to create a Deployment with 3 ReplicaSets. These ReplicaSets are owned by the Deployment and are managed by the Deployment controller. We would also learn how to carry out RollingUpdate and RollBack to new and older versions of the deployment.

In [ ]:
from kubernetes import client, config

### Load config from default location

In [ ]:
config.load_incluster_config()
extension = client.ExtensionsV1beta1Api()

### Create Deployment object

In [ ]:
deployment = client.ExtensionsV1beta1Deployment()

### Fill required Deployment fields (apiVersion, kind, and metadata)

In [ ]:
deployment.api_version = "extensions/v1beta1"
deployment.kind = "Deployment"
deployment.metadata = client.V1ObjectMeta(name="nginx-deployment")

### A Deployment also needs a .spec section

### Add Pod template in .spec.template section

In [ ]:
container = client.V1Container(name="nginx")
container.name="nginx"
container.image="nginx:1.7.9"
container.ports = [client.V1ContainerPort(container_port=80)]

template = client.V1PodTemplateSpec()
template.metadata = client.V1ObjectMeta(labels={"app": "nginx"})
template.spec = client.V1PodSpec(containers=[container])

spec = client.ExtensionsV1beta1DeploymentSpec(template=template)
spec.replicas = 3

### Pod template container description

In [ ]:
deployment.spec = spec

### Create Deployment

In [ ]:
extension.create_namespaced_deployment(namespace="default", body=deployment)

### Update container image 

In [ ]:
deployment.spec.template.spec.containers[0].image = "nginx:1.9.1"

### Apply update (RollingUpdate)

In [ ]:
extension.replace_namespaced_deployment(name="nginx-deployment", namespace="default", body=deployment)

### Create DeploymentRollback object
This object is used to rollback to a previous version of the deployment.

In [ ]:
rollback_to = client.ExtensionsV1beta1RollbackConfig(revision=0)
rollback = client.ExtensionsV1beta1DeploymentRollback(name = "nginx-deployment",
                                                      api_version="extensions/v1beta1",
                                                      kind = "DeploymentRollback",
                                                      rollback_to = rollback_to
                                                      )

### Execute RollBack

In [ ]:
extension.create_namespaced_deployment_rollback(name="nginx-deployment", namespace="default", body=rollback)

### Delete Deployment

In [ ]:
extension.delete_namespaced_deployment(name="nginx-deployment", namespace="default", body=client.V1DeleteOptions(propagation_policy="Foreground", grace_period_seconds=5))